Add location of shared library to search path

In [ ]:
import sys
sys.path.append("../build")

The build places the shared library under 'lib'

In [ ]:
import lib.parquet_table as pqt

In [ ]:
pt = pqt.ParquetTable("../data/20221109.parquet")

In [ ]:
pt.print_stats()

In [ ]:
pt.column(3)

In [ ]:
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
df = pd.DataFrame({ 
    "time": pt.column(0),
    "price": pt.column(3),
    "size": pt.column(2)
})

In [ ]:
df.tail()

In [ ]:
def parse_time(x):
    x = datetime.datetime.fromtimestamp(x/1e6)
    return x.strftime('%b-%d %H:%M')
    
df['time_bin'] = df['time'].apply(parse_time)

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
tf = df.groupby('time_bin').agg({
    'size': np.sum,
    'price': lambda x: x.iloc[-1]
}).reset_index()
tf.head()

In [ ]:
tf.plot(x='time_bin', y=['size'], figsize=(16, 8))

In [ ]:
tf.plot(x='time_bin', y=['price'], figsize=(16, 8))

In [ ]:
from fastparquet import ParquetFile
pf = ParquetFile("../data/20221109.parquet")

In [ ]:
df = pf.to_pandas()